In [1]:
import pandas as pd

In [2]:
rename_columns = {
    "Data_ID": "dataset_id",
    "IndicatorID": "indicator_id",
    "Category": "category",
    "Indicator Name": "name",
    "Data_Name": "name",
    "Indicator_Description": "description",
    "Indicator Source(s)": "source",
    "Dataset_Description": "description",
    "Dataset_Examples_ Citation": "citation",
    "Dataset_Sources_Examples_ Licenses": "licenses",
    "Dataset_Source": "source",
    "Dataset_Examples_ Notes": "notes",
    "Dataset_ Citation": "citation",
    "Dataset_Licenses": "license",
}

# ddf = pd.read_csv("datasets.csv").dropna(how="all").rename(columns=rename_columns).set_index("dataset_id")
# idf = pd.read_csv("indicators.csv").dropna(how="all").rename(columns=rename_columns).set_index("indicator_id")
file = "VALORADA_Data catalogue_CLean_Version_12.06.2025.xlsx"


idf = (
    pd.read_excel(file, sheet_name="Page_1_CIC_Indicators_Data")
    .dropna(how="all")
    .rename(columns=rename_columns)
    .set_index("indicator_id")
)
ddf = (
    pd.read_excel(file, sheet_name="Page_2_Data_To_Indicators")
    .dropna(how="all")
    .rename(columns=rename_columns)
    .set_index("dataset_id")
)

In [10]:
ddf_link_cols = [
    c for c in ddf.columns if c.lower().startswith("ind") and "id" in c.lower()
]
# ddf[[c for c in ddf.columns if c not in link_cols]].to_csv("datasets.csv", index=True)
ddf_links = ddf[
    ddf_link_cols
]  # .reset_index().melt(["dataset_id"], value_name="dataset_id").drop(columns=["variable"])

idf_link_cols = [
    c for c in idf.columns if c.lower().startswith("data_") and "id" in c.lower()
]
# idf[[c for c in idf.columns if c not in link_cols]].to_csv("indicators.csv", index=True)
idf_links = idf[
    idf_link_cols
]  # .reset_index().melt(["indicator_id"], value_name="dataset_id").drop(columns=["variable"])

In [11]:
cols = ["indicator_id", "dataset_id"]

idf_links.reset_index().melt(
    id_vars=["indicator_id"],
    value_vars=idf_link_cols,
    value_name="dataset_id",
).replace("xxx", pd.NA).dropna(subset=["dataset_id"]).drop(columns=["variable"])[
    cols
].sort_values(
    by=["indicator_id", "dataset_id"]
)[
    cols
].to_csv(
    "links_indicator_to_data.csv", index=False
)

In [12]:
ddf_links.reset_index().melt(
    id_vars=["dataset_id"],
    value_vars=ddf_link_cols,
    value_name="indicator_id",
).replace("xxx", pd.NA).dropna(subset=["indicator_id"]).drop(columns=["variable"])[
    cols
].sort_values(
    by=["indicator_id", "dataset_id"]
).to_csv(
    "links_data_to_indicator.csv", index=False
)

In [13]:
ddf[[c for c in ddf.columns if c in rename_columns.values()]].to_csv(
    "datasets.csv", index=True
)
idf[[c for c in idf.columns if c in rename_columns.values()]].to_csv(
    "indicators.csv", index=True
)

In [14]:
# group links for airtable
links = pd.read_csv("links_indicator_to_data.csv")
links = (
    links.groupby("indicator_id")["dataset_id"]
    .apply(lambda x: ",".join(x))
    .reset_index()
)
links.to_csv("links_grouped_indicator_to_data.csv", index=False)

# group links for airtable
links = pd.read_csv("links_data_to_indicator.csv")
links = (
    links.groupby("indicator_id")["dataset_id"]
    .apply(lambda x: ",".join(x))
    .reset_index()
)
links.to_csv("links_grouped_data_to_indicator.csv", index=False)